In [13]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
from sklearn import preprocessing
import numpy as np
import time
import os
from PIL import Image
import json
#from pathlib import Pat

In [14]:
# Replace with valid values
ENDPOINT = "https://0217-1.cognitiveservices.azure.com/"
training_key = "f78fb297caf64e1db1b532723952db92"
prediction_key = "bc05678fd32e4271b396b4938b9315af"
prediction_resource_id = "/subscriptions/8f4b15f6-5895-4afe-93e2-c87116a788a7/resourceGroups/0217test/providers/Microsoft.CognitiveServices/accounts/02171-Prediction"

In [15]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [16]:
publish_iteration_name = "detectModel"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
project = trainer.create_project("My Detection Project", domain_id=obj_detection_domain.id)

Creating project...


In [17]:
# Make two tags in the new project
black_tag = trainer.create_tag(project.id, "black")
white_tag = trainer.create_tag(project.id, "white")

In [18]:
import numpy as np
from sklearn.preprocessing import normalize

black_image_regions_pre= {
    "000125":[131, 394, 375, 624],
    "000126":[110, 0, 443, 322],
    "000127":[134, 69, 369, 311],
    "000128":[60, 149, 364, 503],
    "000129":[0, 0, 467, 400],
    "000130":[2, 86, 467, 503],
    "000131": [75, 184, 410, 476],
    "000132":[0, 0, 434, 453], 
    "000133":[347, 190, 704, 554],
    "000134":[127, 262, 656, 741],
    "000741": [1, 1, 467, 620], 
    "000742": [199, 381, 403, 852], 
    "000743": [243, 389, 406, 863], 
    "000744": [148, 15, 329, 611], 
    "000745": [197, 388, 383, 870]
}


black_image_regions= {
    "000125":[ 0.145833328, 0.3509314, 0.5894608, 0.238562092 ],
    "000126": [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000127": [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000128": [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000129": [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000130": [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000131":  [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000132": [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000133": [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000134": [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000741":  [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000742":  [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000743":  [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000744":  [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "000745":  [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ]
}





#print(type(black_image_regions_pre.values()))


# white_image_regions = {
#     "000235":  [ 0.2365196, 0.128709182, 0.5845588, 0.71405226 ],
#     "000236":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
#     "000237":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ], 
#     "000238":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
#     "000239":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
#     "000240":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
#     "000241":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
#     "000242":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
#     "000243":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
#     "000244":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
#     "000245":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ], 
#     "000246":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ], 
#     "000247":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
#     "000248":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ], 
#     "000249":[ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ]
    
    
# }


white_image_regions_pre = {
    "000235": [2, 0, 466, 625], 
    "000236":[2, 1, 467, 348],
    "000237":[0, 4, 466, 625], 
    "000238":[2, 6, 467, 626], 
    "000239":[2, 81, 467, 626],
    "000240":[0, 4, 466, 665], 
    "000241":[334, 129, 491, 375],
    "000242":[258, 424, 635, 818], 
    "000243":[273, 325, 560, 637],
    "000244":[259, 82, 418, 403],
    "000245":[243, 339, 492, 690], 
    "000246":[279, 358, 442, 679], 
    "000247":[283, 364, 478, 680],
    "000248":[344, 331, 531, 636], 
    "000249":[319, 326, 548, 650]
    
    
}

    

In [19]:

#建一個w_json的資料夾 裡面放圖片跟json檔
json_p = 'C:/Users/JIALI/Desktop/w_json/'
jList = os.listdir(json_p)
white_image_regions={} #這是一個dic的資料型態
        
for file in jList:
    
    if file.endswith(".jpg"):
        img = Image.open( os.path.join(json_p,file))
        imgSize = img.size  #大小/尺寸
        pw = img.width       #图片的宽
        ph = img.height      #图片的高
        pf = img.format      #图像格式
      #  print(imgSize)
        #print(pw, ph, pf)
    
# Region Left(rl) = X1 / Image Width
# Region Top(rt) = Y1 / Image Height
# Region Width(rw) = (X2 — X1) / image Width
# Region Height(rh)= (Y2 — Y1) / image Height
    
    if file.endswith(".json"):
        with open(os.path.join(json_p,file) , 'r') as reader:
            jf = json.loads(reader.read())
        if jf['item1']['category_id']==2:#如果種類是上衣
            print(jf['item1']['bounding_box'])
           # print(jf['item1']['bounding_box'][0])
            rl=jf['item1']['bounding_box'][0]/pw
            rt = jf['item1']['bounding_box'][1]/ph
            rw=(jf['item1']['bounding_box'][2]-jf['item1']['bounding_box'][0])/pw
            rh=(jf['item1']['bounding_box'][3]-jf['item1']['bounding_box'][1])/ph
            #print(Path(file).stem)
            white_image_regions[Path(file).stem]=[rl,rt,rw,rh]  #把東西丟到這個dic裡         
          #  print (white_image_regions)
  
    

[2, 0, 466, 625]
[2, 1, 467, 348]
[0, 4, 466, 625]
[2, 6, 467, 626]
[2, 81, 467, 626]
[0, 4, 466, 665]
[334, 129, 491, 375]
[258, 424, 635, 818]
[273, 325, 560, 637]
[259, 82, 418, 403]
[243, 339, 492, 690]
[279, 358, 442, 679]
[283, 364, 478, 680]
[344, 331, 531, 636]
[319, 326, 548, 650]


In [20]:
# import os
# from PIL import Image
# file_path = 'C:/Users/JIALI/Desktop/black/'
# allFileList = os.listdir(file_path)
        
# for file in allFileList:
   
#     img = Image.open( os.path.join(file_path,file))
#     imgSize = img.size  #大小/尺寸
#     w = img.width       #图片的宽
#     h = img.height      #图片的高
#     f = img.format      #图像格式
    

#     print(imgSize)
#     print(w, h, f)

    

    

In [21]:
# Update this with the path to where you downloaded the images.
base_image_location = "C:/Users/JIALI/Desktop/"

# Go through the data table above and create the images
print ("Adding images...")
tagged_images_with_regions = []

for file_name in black_image_regions.keys():
    x,y,w,h = black_image_regions[file_name]
    regions = [ Region(tag_id=black_tag.id, left=x,top=y,width=w,height=h) ]

    with open(base_image_location + "black/" + file_name + ".jpg", mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

for file_name in white_image_regions.keys():
    x,y,w,h = white_image_regions[file_name]
    regions = [ Region(tag_id=white_tag.id, left=x,top=y,width=w,height=h) ]

    with open(base_image_location + "white/" + file_name + ".jpg", mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))

upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=tagged_images_with_regions))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

Adding images...


In [22]:
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training


In [11]:
# The iteration is now trained. Publish it to the project endpoint
#trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
trainer.publish_iteration(project.id, iteration.id,publish_iteration_name ,  prediction_resource_id)

print ("Done!")

CustomVisionErrorException: Invalid publish target. A valid publish target should look like: /subscriptions/{subscription id}/resourceGroups/{resource group name}/providers/Microsoft.CognitiveServices/accounts/{prediction resource name}

In [ ]:
# Now there is a trained endpoint that can be used to make a prediction

# Open the sample image and get back the prediction results.
with open(base_image_location + "test/000024.jpg", mode="rb") as test_data:
    results = predictor.detect_image(project.id, "iteration-1", test_data)

# Display the results.    
for prediction in results.predictions:
    print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))